In [1]:
from line_solver import *
import numpy as np

GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

n = 5  # number of items
m = 2  # cache capacity

source = Source(model, 'Source')
cacheNode = Cache(model, 'Cache', n, m, ReplacementStrategy.RR)
sink = Sink(model, 'Sink')

jobClass = OpenClass(model, 'InitClass', 0)
hitClass = OpenClass(model, 'HitClass', 0)
missClass = OpenClass(model, 'MissClass', 0)

source.set_arrival(jobClass, Exp(2))

#pAccess = DiscreteSampler((1/n)*ones(1,n))  # uniform item references
pAccess = Zipf(1.4, n)  # Zipf-like item references
cacheNode.set_read(jobClass, pAccess)

cacheNode.set_hit_class(jobClass, hitClass)
cacheNode.set_miss_class(jobClass, missClass)

P = model.init_routing_matrix()
P.set(jobClass, jobClass, source, cacheNode, 1.0)
P.set(hitClass, hitClass, cacheNode, sink, 1.0)
P.set(missClass, missClass, cacheNode, sink, 1.0)

model.link(P)

In [3]:
solver = np.empty(4, dtype=object)
AvgTable = np.empty(4, dtype=object)

solver[0] = CTMC(model, keep=False, cutoff=1, seed=1)
AvgTable[0] = solver[0].avg_node_table()

CTMC analysis [method: default, lang: java, env: 21.0.8] completed in 0.156982s.
     Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0  Source  InitClass   0.0   0.0    0.0     0.0   0.0   2.0
3   Cache  InitClass   0.0   0.0    0.0     0.0   2.0   0.0
4   Cache   HitClass   0.0   0.0    0.0     0.0   0.0   2.0
7    Sink   HitClass   0.0   0.0    0.0     0.0   2.0   0.0


In [4]:
model.reset()
solver[1] = SSA(model, samples=10000, verbose=True, method="serial", seed=1)
AvgTable[1] = solver[1].avg_node_table()

SSA samples:   10000
SSA analysis [method: serial, lang: java, env: 21.0.8] completed in 0.728864s. Iterations: 10001.
     Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0  Source  InitClass   0.0   0.0    0.0     0.0  0.0000  2.0000
3   Cache  InitClass   0.0   0.0    0.0     0.0  2.0000  0.0000
4   Cache   HitClass   0.0   0.0    0.0     0.0  0.0000  1.1633
5   Cache  MissClass   0.0   0.0    0.0     0.0  0.0000  0.8367
7    Sink   HitClass   0.0   0.0    0.0     0.0  1.1633  0.0000
8    Sink  MissClass   0.0   0.0    0.0     0.0  0.8367  0.0000


In [5]:
model.reset()
solver[2] = MVA(model, seed=1)
AvgTable[2] = solver[2].avg_node_table()

MVA analysis [method: default(fpi), lang: java, env: 21.0.8] completed in 0.004558s.
     Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0  Source  InitClass   0.0   0.0    0.0     0.0  0.0000  2.0000
3   Cache  InitClass   0.0   0.0    0.0     0.0  2.0000  0.0000
4   Cache   HitClass   0.0   0.0    0.0     0.0  0.0000  1.1127
5   Cache  MissClass   0.0   0.0    0.0     0.0  0.0000  0.8873
7    Sink   HitClass   0.0   0.0    0.0     0.0  1.1127  0.0000
8    Sink  MissClass   0.0   0.0    0.0     0.0  0.8873  0.0000


In [6]:
model.reset()
solver[3] = NC(model, seed=1)
AvgTable[3] = solver[3].avg_node_table()

hitRatio = cacheNode.hit_ratio()
missRatio = cacheNode.miss_ratio()
print(f"hitRatio = {hitRatio}")
print(f"missRatio = {missRatio}")

NC analysis [method: default(rayint), lang: java, env: 21.0.8] completed in 0.033308s. Iterations: 100000.
     Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0  Source  InitClass   0.0   0.0    0.0     0.0  0.0000  2.0000
3   Cache  InitClass   0.0   0.0    0.0     0.0  2.0000  0.0000
4   Cache   HitClass   0.0   0.0    0.0     0.0  0.0000  1.1722
5   Cache  MissClass   0.0   0.0    0.0     0.0  0.0000  0.8278
7    Sink   HitClass   0.0   0.0    0.0     0.0  1.1722  0.0000
8    Sink  MissClass   0.0   0.0    0.0     0.0  0.8278  0.0000
hitRatio = [[0.58609163 0.         0.        ]]
missRatio = [[0.41390837 1.         1.        ]]
